<a href="https://colab.research.google.com/github/Stephen42Kim/IHS/blob/main/Top50MSASpectralClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests                   # Make HTTP requests
import json                       # Load US Census datasets as JSON
import numpy as np
import pandas as pd
from google.colab import files    # Access local directory
#from google.colab import drive   # Access Google Drive (in folder /content/drive/My Drive/IHS/)
#drive.mount('/content/drive')

# IHS Urban Spectral Clustering On Top 50 MSAs

